In [1]:
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
import cv2
import os

In [60]:
def read_data(data_paths, max_images_per_directory=100, resize_shape=(255, 255)):
    dataset = []
    labels = []
    j = 0
    classes = []
    for data_path in data_paths:
        DIRPATH = os.listdir(data_path)
        for i in range(len(DIRPATH)):
            current_dir_path = os.path.join(data_path, DIRPATH[i])
            images_in_directory = os.listdir(current_dir_path)[:max_images_per_directory]

            for img in images_in_directory:
                image_path = os.path.join(current_dir_path, img)
                image = np.mean(cv2.imread(image_path), axis=-1) / 255.0  # Read as grayscale and normalize
                # Resize the image to a consistent shape
                image = cv2.resize(image, resize_shape)
                dataset.append(image)
                labels.append(j)
            j+=1
        classes.extend(DIRPATH)
        
    return np.array(dataset, dtype=np.float32), np.array(labels, dtype=np.int64), classes

In [61]:
breast_cancer = r'/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT'
fracture = r'/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/val'
brain_tumor = r'/kaggle/input/brain-tumor-classification-mri/Training'
pneumonia = r'/kaggle/input/chest-xray-pneumonia/chest_xray/test'
kidney = '/kaggle/input/ct-kidney-dataset-normal-cyst-tumor-and-stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone'

In [54]:
data_paths = [breast_cancer, fracture, brain_tumor, pneumonia, kidney]
dataset, labels, classes = read_data(data_paths)

In [59]:
print(set(labels))
print(classes)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
['benign', 'normal', 'malignant', 'not fractured', 'fractured', 'no_tumor', 'pituitary_tumor', 'meningioma_tumor', 'glioma_tumor', 'PNEUMONIA', 'NORMAL', 'Cyst', 'Tumor', 'Stone', 'Normal']


In [64]:
dataset, labels = shuffle(dataset, labels)
labels[0], dataset[0].shape

(3, (255, 255))

In [65]:
training_data, testing_data = dataset[:int(len(dataset)*0.9)], dataset[int(len(dataset)*0.9)+1:]
training_labels, testing_labels = labels[:int(len(labels)*0.9)], labels[int(len(labels)*0.9)+1:]

In [66]:
training_labels[:5]

array([ 3,  9,  4, 14,  6])

In [67]:
print(type(training_data), type(testing_labels))
print(training_data.shape, testing_labels.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(1350, 255, 255) (149,)


In [85]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(255, 255, 1)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(15, activation='softmax'))

In [90]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [91]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 253, 253, 32)      320       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 126, 126, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 124, 124, 32)      9248      
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 123008)            0         
                                                                 
 dense_8 (Dense)             (None, 64)               

In [92]:
model.fit(training_data, training_labels, shuffle=True, validation_data=(testing_data, testing_labels), epochs=10, batch_size=64)

Epoch 1/10
22/22 [==============================] - 53s 2s/step - loss: 2.3615 - accuracy: 0.2607 - val_loss: 1.3801 - val_accuracy: 0.4966
Epoch 2/10
22/22 [==============================] - 52s 2s/step - loss: 1.0175 - accuracy: 0.6304 - val_loss: 0.8499 - val_accuracy: 0.6577
Epoch 3/10
22/22 [==============================] - 52s 2s/step - loss: 0.5626 - accuracy: 0.7844 - val_loss: 0.6653 - val_accuracy: 0.7651
Epoch 4/10
22/22 [==============================] - 53s 2s/step - loss: 0.3397 - accuracy: 0.8756 - val_loss: 0.6058 - val_accuracy: 0.8389
Epoch 5/10
22/22 [==============================] - 53s 2s/step - loss: 0.1847 - accuracy: 0.9326 - val_loss: 0.7017 - val_accuracy: 0.7785
Epoch 6/10
22/22 [==============================] - 52s 2s/step - loss: 0.1173 - accuracy: 0.9622 - val_loss: 0.7593 - val_accuracy: 0.7584
Epoch 7/10
22/22 [==============================] - 52s 2s/step - loss: 0.0604 - accuracy: 0.9837 - val_loss: 0.7445 - val_accuracy: 0.7718
Epoch 8/10
22/22 [==

In [94]:
model.save('global_model.h5')